# PACKAGE IMPORTS

In [ ]:
# Standard library imports
import os
import re
import warnings
from math import sqrt

# Data science and visualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Scientific computing
from scipy import stats

# Statistics and econometrics
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox, het_arch, het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tsa.api import VAR

# Machine learning
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Suppress warnings
warnings.filterwarnings("ignore")

In [ ]:
DAILY_YIELDS_CSV = "files/raw/us_treasury_yields_daily.csv"
YIELDS_CSV = "files/raw/yields.csv"
MACRO_CSV  = "files/raw/world_bank_development_indicators.csv"
WORLD_BANK_DATA = "files/raw/world_bank_data_2025.csv"
DGS10 = "files/raw/DGS10.csv"
IRLTLT="files/raw/IRLTLT01DEM156N.csv"

# USA: 10Y Yield vs Inflation

##### Functions

In [ ]:
# columna 10 años
def detect_10y_col(cols):
    for c in cols:
        cl = c.strip().lower()
        if cl in {"us10y","10y","10 yr","10-year","10 year","10_yr"}:
            return c
    for c in cols:
        cl = c.lower()
        if "10" in cl and ("yr" in cl or "year" in cl):
            return c
    if "US10Y" in cols: return "US10Y"
    if "10 Yr" in cols: return "10 Yr"
    raise ValueError(f"No encuentro columna 10Y en yields. Columnas: {list(cols)[:12]}")

In [ ]:
def extract_year_col(df, preferred_cols=("year","date","fecha","period","time","obs_date","observationdate")):
    """
    Devuelve una Serie 'Year' (int) extraída de la mejor columna disponible:
    - Si hay 'year' numérico, lo usa.
    - Si hay fechas, parsea a datetime y saca el año.
    - Si hay strings tipo '1960 [YR1960]' o 'YR1960', extrae los 4 dígitos por regex.
    """
    # 1) columna explícita de año
    for c in df.columns:
        if c.lower() in ("year","anio","año"):
            y = pd.to_numeric(df[c], errors="coerce")
            if y.notna().any():
                return y.astype("Int64")
    # 2) columnas “fecha”
    for cand in preferred_cols:
        for c in df.columns:
            if c.lower()==cand:
                # a) intentar datetime
                dt = pd.to_datetime(df[c], errors="coerce", dayfirst=False)
                if dt.notna().any():
                    return dt.dt.year.astype("Int64")
                # b) extraer patrón de 4 dígitos
                s = df[c].astype(str).str.extract(r'(\d{4})', expand=False)
                y = pd.to_numeric(s, errors="coerce")
                if y.notna().any():
                    return y.astype("Int64")
    # 3) si ninguna funciona, intentar extraer 4 dígitos de cualquier columna de texto
    for c in df.columns:
        if df[c].dtype==object:
            s = df[c].astype(str).str.extract(r'(\d{4})', expand=False)
            y = pd.to_numeric(s, errors="coerce")
            if y.notna().sum() >= len(df)*0.3:
                return y.astype("Int64")
    return pd.Series([pd.NA]*len(df), index=df.index, dtype="Int64")

In [ ]:
def find_inflation_col(df):
    """
    Busca una columna de inflación (%). Prioriza nombres típicos.
    """
    prefs = ["inflation_annual%", "inflation_yoy", "inflation yoy", "inflation%", "inflation", "cpi_yoy", "cpi yoy"]
    # prioridad por nombre
    for p in prefs:
        for c in df.columns:
            if p.replace(" ","") in c.replace(" ","").lower():
                return c
    # si no, una 'value' genérica
    for c in df.columns:
        if "value" in c.lower() or "valor" in c.lower():
            return c
    # último recurso: columna numérica “prometedora”
    numc = [c for c in df.columns if pd.to_numeric(df[c], errors="coerce").notna().sum()>0]
    if numc:
        return numc[0]
    return None

In [ ]:
def filter_country_usa(df):
    """
    Intenta filtrar USA por varias columnas posibles.
    """
    # a) 'Country Code' == USA
    for c in df.columns:
        if c.lower() in ("country code","country_code","iso3","isocode","code"):
            mask = df[c].astype(str).str.upper().eq("USA")
            if mask.any(): 
                return df.loc[mask].copy()
    # b) 'Country' / 'Country Name' contiene 'United States' o 'USA'
    for c in df.columns:
        if c.lower() in ("country","country name","pais"):
            mask = df[c].astype(str).str.contains(r"\b(united states|usa)\b", case=False, na=False)
            if mask.any():
                return df.loc[mask].copy()
    # si no hay país, asumimos que el archivo ya es solo USA (devolver tal cual)
    return df.copy()

##### Code

In [ ]:

# (funciona con "country|date|inflation_annual%")

# ---------- 1) Yields USA -> anual 10Y ----------
y = pd.read_csv(DAILY_YIELDS_CSV)

# fecha
date_col = next((c for c in y.columns if c.lower() in {"date","fecha"}), None)
if date_col is None:
    raise ValueError("No encuentro columna de fecha en yields CSV.")
y[date_col] = pd.to_datetime(y[date_col], errors="coerce")
y = y.dropna(subset=[date_col])
col10 = detect_10y_col(y.columns)

# anual
y = y[[date_col, col10]].rename(columns={date_col:"date", col10:"yield_10y"})
y["Year"] = y["date"].dt.year.astype(int)
us_annual = y.groupby("Year", as_index=False)["yield_10y"].mean()
print("YIELDS USA — años:", us_annual["Year"].min(), "→", us_annual["Year"].max(), "| n:", len(us_annual))

# ========= DIAGNÓSTICO + EXTRACCIÓN ROBUSTA DE INFLACIÓN (USA) =========
m = pd.read_csv(MACRO_CSV)

# 1) filtrar USA
m_usa = filter_country_usa(m)

# 2) obtener Year robusto
m_usa["Year"] = extract_year_col(m_usa)

# 3) localizar columna de inflación
infl_col = find_inflation_col(m_usa)
if infl_col is None:
    raise RuntimeError("No encuentro columna de inflación en tu archivo macro. Revisa nombres de columnas.")

# 4) preparar inflación anual
infl = (
    m_usa[["Year", infl_col]]
      .rename(columns={infl_col:"inflation_yoy"})
      .assign(inflation_yoy=lambda d: pd.to_numeric(d["inflation_yoy"], errors="coerce"))
      .dropna(subset=["Year","inflation_yoy"])
)
infl["Year"] = infl["Year"].astype(int)

# Si es mensual (múltiples filas por año), promediamos
infl = infl.groupby("Year", as_index=False)["inflation_yoy"].mean()

print("Yields USA: ", us_annual["Year"].min(), "→", us_annual["Year"].max(), "| n:", len(us_annual))
print("Inflación USA:", infl["Year"].min() if len(infl) else None, "→", infl["Year"].max() if len(infl) else None, "| n:", len(infl))

# 5) forzar intersección y MERGE
min_year = max(us_annual["Year"].min(), infl["Year"].min())
max_year = min(us_annual["Year"].max(), infl["Year"].max())
infl_clip = infl[(infl["Year"]>=min_year) & (infl["Year"]<=max_year)]
ua_clip   = us_annual[(us_annual["Year"]>=min_year) & (us_annual["Year"]<=max_year)]

df = ua_clip.merge(infl_clip, on="Year", how="inner").sort_values("Year").reset_index(drop=True)
if df.empty:
    # diagnóstico extra
    print(">>> DEBUG — YEARS YIELDS:", sorted(us_annual["Year"].unique())[:10], "...", sorted(us_annual["Year"].unique())[-10:])
    print(">>> DEBUG — YEARS INFL :", sorted(infl["Year"].unique())[:10], "...", sorted(infl["Year"].unique())[-10:])
    raise RuntimeError("Sigue sin haber años comunes. Revisa que 'Year' se esté extrayendo bien del macro CSV.")

df["real_yield"] = df["yield_10y"] - df["inflation_yoy"]

print("Filas tras merge:", len(df))
print(df.head())


In [ ]:
plt.figure(figsize=(10,5))
plt.plot(df["Year"], df["yield_10y"], label="US 10Y Yield")
plt.plot(df["Year"], df["inflation_yoy"], label="Inflation YoY")
plt.title("USA: 10Y Yield vs Inflation")
plt.xlabel("Year"); plt.ylabel("%"); plt.legend(); plt.tight_layout(); plt.show()

plt.figure(figsize=(6,5))
plt.scatter(df["inflation_yoy"], df["yield_10y"])
plt.title("Relación: Inflación vs US 10Y Yield")
plt.xlabel("Inflation YoY (%)"); plt.ylabel("US 10Y Yield (%)")
plt.tight_layout(); plt.show()


#### 1. Serie temporal: USA 10Y Yield vs Inflación (líneas)

La línea azul (US 10Y Yield) muestra la rentabilidad de los bonos a 10 años.

La línea naranja (Inflation YoY) muestra la inflación anual.

Interpretación rápida:

En los años 70-80 se ve un pico muy alto de inflación y yield: es la época de la crisis del petróleo y la política monetaria dura de la Fed (Volcker subió tipos muy fuertes).

Desde los años 90 hasta 2020, ambos bajan gradualmente → entorno de baja inflación y yields decrecientes.

Después de 2020, la inflación se dispara otra vez (post-COVID, guerra, disrupciones en energía).

👉 Eso muestra que existe cierta relación: cuando la inflación sube fuerte, los yields suelen subir también, pero con rezagos o diferente magnitud.

#### 2. Diagrama de dispersión: Inflación vs Yield

Cada punto es un año (x = inflación, y = yield).

Se ve una nube ascendente: cuando la inflación es baja (0-4%), los yields tienden a estar en 2-6%.

Cuando la inflación es alta (6-12%), los yields también tienden a estar arriba (8-14%).

Interpretación rápida:

Hay una correlación positiva: inflación y yields tienden a moverse juntos.

Pero no es 1:1 → hay dispersión (porque los yields dependen también de política monetaria, expectativas, riesgo, etc.).

#### 3. Resumen antes de limpieza

Los datos ya muestran una relación clara: más inflación = más yield.

Pero hay ruido y valores extremos (ej: hiperinflación o datos atípicos que vimos en la tabla).

Por eso, la siguiente etapa de limpieza (quitar inflaciones absurdas, errores, etc.) servirá para tener relaciones más nítidas y modelos más confiables.

### Estadísticas básicas antes de limpiar


In [ ]:

print("\n--- Correlación simple ---")
print(df[["yield_10y", "inflation_yoy"]].corr())

# Regresión lineal simple (Inflación -> Yield)

X = df["inflation_yoy"]
y = df["yield_10y"]

X = sm.add_constant(X)  # añadimos intercepto
model = sm.OLS(y, X).fit()

print("\n--- Regresión lineal simple ---")
print(model.summary())


### 📊 Correlación simple

0.61 (positivo, moderado-fuerte) → Significa que cuando la inflación sube, los rendimientos del bono a 10 años en USA tienden a subir también.

No es una correlación perfecta (1.0), pero sí clara y estadísticamente significativa.

#### 📈 Regresión lineal simple

Modelo:

Yield 10Y

=
3.3783
+
0.6571
⋅
Inflaci
o
ˊ
n YoY
Yield 10Y=3.3783+0.6571⋅Inflaci
o
ˊ
n YoY

Constante (3.38): cuando la inflación es 0, el modelo predice un rendimiento del 3.38%.

Coeficiente (0.6571): por cada +1% en la inflación, el yield sube en promedio +0.65%.

p-valor (0.000): relación estadísticamente muy significativa.

R² = 0.38: la inflación explica el 38% de la variación en los yields.

Esto es bastante alto para datos macro, pero también nos dice que hay un 62% de variación que se explica por otros factores (ej. política monetaria, riesgo país, oferta/demanda global de bonos, etc.).

#### 📌 Conclusión preliminar

Existe una relación positiva clara entre inflación y yields en USA (1962–2024).

La inflación no lo explica todo, pero sí es un driver muy importante.

Esto tiene sentido económico: los inversores piden más rentabilidad en los bonos cuando esperan más inflación (para no perder poder adquisitivo).

✅ Resumen rápido (checkpoint hasta ahora):

Ya cargamos y visualizamos las series de rendimiento del bono USA 10Y y inflación anual (1962–2024).

Encontramos una correlación positiva moderada (0.61) → cuando la inflación sube, los yields también tienden a subir.

La regresión lineal mostró que un +1% en inflación se asocia con un +0.65% en el yield 10Y, con R² = 0.38 → la inflación explica parte importante, pero no todo.

In [ ]:

# Paso 1: Copiar y limpiar outliers
# ===============================

df_clean = df.copy()

# Quitamos inflaciones absurdas (ej. errores de dataset o hiperinflación)
df_clean = df_clean[df_clean["inflation_yoy"].between(-20, 50)]

print("Filas antes:", len(df), " | después:", len(df_clean))

# ===============================
# Paso 2: Recalcular real_yield
# ===============================

df_clean["real_yield"] = df_clean["yield_10y"] - df_clean["inflation_yoy"]

# ===============================
# Paso 3: Chequeo rápido
# ===============================
print("\nÚltimos años limpios:")
print(df_clean.tail(10))

# ===============================
# Paso 4: Visualización post-limpieza
# ===============================


plt.figure(figsize=(10,5))
plt.plot(df_clean["Year"], df_clean["yield_10y"], label="US 10Y Yield")
plt.plot(df_clean["Year"], df_clean["inflation_yoy"], label="Inflation YoY")
plt.plot(df_clean["Year"], df_clean["real_yield"], label="Real Yield", linestyle="--")
plt.title("USA: Yield 10Y, Inflación y Real Yield (datos limpios)")
plt.xlabel("Year")
plt.ylabel("%")
plt.legend()
plt.tight_layout()
plt.show()


### Interpretación rápida del gráfico

Años 70s – 80s:

La inflación se dispara (choques del petróleo).

Los rendimientos nominales (azul) también suben fuerte.

El rendimiento real (verde) se mantiene positivo pero muy volátil.

Años 90s – 2000s:

Inflación baja y estable (2–3%).

Yield nominal cae gradualmente.

El rendimiento real es bajo pero estable.

2010s en adelante:

Rendimientos muy bajos (política monetaria expansiva).

Inflación estable… hasta el repunte 2021–2022.

En 2021–2022 el real yield se hace negativo (bonos pierden contra inflación).

#### Qué significa:

Cuando el real yield es positivo → los bonos dan un retorno por encima de la inflación (buen refugio).

Cuando el real yield es negativo → los inversores pierden poder adquisitivo aunque inviertan en bonos.

#### Correlación con datos limpios ---


In [ ]:
print("\n--- Correlación simple (datos limpios) ---")
print(df_clean[["yield_10y", "inflation_yoy"]].corr())


#### Regresión lineal simple (datos limpios)


In [ ]:
X = df_clean[["inflation_yoy"]]
y = df_clean["yield_10y"]

# Agregar constante al modelo
X = sm.add_constant(X)

model_clean = sm.OLS(y, X).fit()
print("\n--- Regresión lineal simple (datos limpios) ---")
print(model_clean.summary())


#### Interpretación del output de la regresión lineal:

Coeficientes (const y inflación)

Constante (const = 3.3783) → cuando la inflación es 0, el rendimiento de los bonos a 10 años tiende a estar en torno al 3.38%.

Inflación (0.6571) → por cada +1% en inflación, el yield sube en promedio +0.66 puntos porcentuales. Esto confirma una relación positiva clara.

R² (0.380)

El 38% de la variabilidad del rendimiento de los bonos a 10 años se explica por la inflación.

Esto significa que la inflación influye mucho, pero no lo explica todo (hay otros factores: política monetaria, expectativas, prima de riesgo, etc.).

p-values

Ambos coeficientes (constante e inflación) tienen p < 0.001, es decir, son estadísticamente significativos.

La relación no es casualidad: hay evidencia fuerte de que la inflación impacta en el yield.

Durbin-Watson (0.227)

Valor bajo → indica que puede haber autocorrelación en los errores (normal en series temporales).

Esto nos dice que quizá más adelante necesitemos un modelo de series temporales (ARIMA, VAR) o un modelo con variables adicionales.

#### Conclusión rápida para tu proyecto
👉 Existe una relación positiva y estadísticamente significativa entre la inflación y el rendimiento del bono USA 10Y: cuando la inflación sube, el yield también sube, aunque la inflación explica solo un 38% de la variación.

### ispersión inflación vs yield con la recta de regresión

In [ ]:

# Scatter plot con la recta de regresión
plt.figure(figsize=(8,6))
sns.scatterplot(x="inflation_yoy", y="yield_10y", data=df_clean, color="blue", label="Datos")

# Ajustar la recta
coef = 0.6571
intercept = 3.3783
x_vals = np.linspace(df_clean["inflation_yoy"].min(), df_clean["inflation_yoy"].max(), 100)
y_vals = intercept + coef * x_vals
plt.plot(x_vals, y_vals, color="red", linewidth=2, label="Recta de regresión")

# Títulos
plt.title("Inflación vs US 10Y Yield con regresión lineal")
plt.xlabel("Inflación YoY (%)")
plt.ylabel("US 10Y Yield (%)")
plt.legend()
plt.show()


### ecuación y el R²

In [ ]:
# Scatter plot con recta de regresión y ecuación
plt.figure(figsize=(8,6))
sns.scatterplot(x="inflation_yoy", y="yield_10y", data=df_clean, color="blue", label="Datos")

# Recta de regresión
coef = 0.6571
intercept = 3.3783
r2 = 0.38  # R-cuadrado de tu modelo
x_vals = np.linspace(df_clean["inflation_yoy"].min(), df_clean["inflation_yoy"].max(), 100)
y_vals = intercept + coef * x_vals
plt.plot(x_vals, y_vals, color="red", linewidth=2, label="Recta de regresión")

# Texto con ecuación
plt.text(0.05, 0.95,
         f"Yield = {intercept:.2f} + {coef:.2f}*Inflation\nR² = {r2:.2f}",
         transform=plt.gca().transAxes,
         fontsize=12, color="red", verticalalignment="top")

# Títulos y etiquetas
plt.title("Inflación vs US 10Y Yield con regresión lineal")
plt.xlabel("Inflación YoY (%)")
plt.ylabel("US 10Y Yield (%)")
plt.legend()
plt.show()


### Diagnóstico del modelo (OLS)


In [ ]:

# 1) Recalcular (por si abriste sesión nueva)
X = sm.add_constant(df_clean[["inflation_yoy"]])
y = df_clean["yield_10y"]
model_clean = sm.OLS(y, X).fit()

# 2) Residuales y ajustados
fitted = model_clean.fittedvalues
resid   = model_clean.resid
std_res = (resid - resid.mean()) / resid.std(ddof=1)

# 3) Plots básicos
plt.figure(figsize=(8,5))
plt.scatter(fitted, resid, alpha=0.8)
plt.axhline(0, linestyle="--")
plt.xlabel("Fitted (predichos)")
plt.ylabel("Residuos")
plt.title("Residuos vs Predichos")
plt.tight_layout(); plt.show()

plt.figure(figsize=(8,5))
plt.hist(resid, bins=12, edgecolor="k", alpha=0.8)
plt.title("Histograma de residuos")
plt.xlabel("Residuo"); plt.ylabel("Frecuencia")
plt.tight_layout(); plt.show()

sm.qqplot(resid, line="s")
plt.title("QQ-plot de residuos")
plt.tight_layout(); plt.show()

# Residuos a lo largo del tiempo (para ver autocorrelación visual)
plt.figure(figsize=(10,4))
plt.plot(df_clean["Year"], resid, marker="o")
plt.axhline(0, linestyle="--")
plt.title("Residuos en el tiempo")
plt.xlabel("Año"); plt.ylabel("Residuo")
plt.tight_layout(); plt.show()

# 4) Tests estadísticos
print("\n--- Tests de diagnóstico ---")

# Normalidad (Jarque–Bera)
try:
    # statsmodels siempre devuelve (jb, pvalue, skew, kurtosis)
    jb_stat, jb_p, jb_skew, jb_kurt = sm_jb(resid)
except Exception:
    # fallback: SciPy (puede devolver solo 2 valores)
    jb = stats.jarque_bera(resid)
    if hasattr(jb, "__len__") and len(jb) >= 2:
        jb_stat, jb_p = jb[0], jb[1]
        jb_skew = jb_kurt = float("nan")
    else:
        jb_stat = float(jb)
        jb_p = float("nan")
        jb_skew = jb_kurt = float("nan")
print(f"Jarque-Bera: stat={jb_stat:.3f}, p-value={jb_p:.4f}  (skew={jb_skew:.3f}, kurt={jb_kurt:.3f})")

# Heterocedasticidad (Breusch–Pagan)
bp_stat, bp_p, _, _ = het_breuschpagan(resid, X)
print(f"Breusch-Pagan: stat={bp_stat:.3f}, p-value={bp_p:.4f}")

# Autocorrelación en residuos (Ljung–Box)
lb = acorr_ljungbox(resid, lags=[1, 4, 8, 12], return_df=True)
print("\nLjung-Box (p-values):")
print(lb["lb_pvalue"].rename(index={1:"lag1",4:"lag4",8:"lag8",12:"lag12"}))

# Durbin–Watson
print(f"\nDurbin-Watson: {durbin_watson(resid):.3f}")

#### 1. Normalidad de los residuos (Jarque–Bera)

JB = 3.936, p-value = 0.1397

Como p > 0.05 → no rechazamos la hipótesis nula de normalidad.
✅ Los residuos son aproximadamente normales → buen síntoma.

2. Heterocedasticidad (Breusch–Pagan)

BP = 1.790, p-value = 0.1809

Como p > 0.05 → no hay evidencia fuerte de heterocedasticidad.
✅ La varianza de los errores parece estable → los errores estándar son confiables.

3. Autocorrelación (Ljung–Box + Durbin-Watson)

Ljung–Box:

lag1, lag4, lag8, lag12 → p-values casi cero → rechazamos la hipótesis de “no autocorrelación”.
❌ Hay autocorrelación muy fuerte en los residuos.

Durbin–Watson = 0.227

Valores cercanos a 2 = no autocorrelación.

Valores < 1 = fuerte autocorrelación positiva.

Aquí 0.227 → autocorrelación positiva extrema.

#### En resumen:

Normalidad: bien.

Homocedasticidad: bien.

Autocorrelación: muy mal → el modelo lineal clásico OLS no captura la dinámica temporal.

👉 Esto significa que tu modelo simple (Yield ~ Inflación) ignora la dependencia temporal. Los rendimientos de bonos son series temporales, por eso los residuos siguen un patrón.

#### Reestimación con errores robustos HAC (Newey-West)

In [ ]:

X = sm.add_constant(df_clean[["inflation_yoy"]])
y = df_clean["yield_10y"]

# Modelo OLS
ols_model = sm.OLS(y, X).fit()

# Ajuste con errores robustos HAC (Newey-West)
# lag=4 (aprox. autocorrelación hasta 4 rezagos, puedes probar otros valores)
nw_model = ols_model.get_robustcov_results(cov_type="HAC", maxlags=4)

print("\n--- Regresión OLS con errores HAC (Newey-West) ---")
print(nw_model.summary())


 

#### 👉 ¿Qué cambió con HAC?

Coeficientes (constante = 3.37, inflación = 0.65) → no cambiaron, porque la relación base sigue siendo la misma.

Errores estándar (y por tanto t-stats y p-values) → son más grandes que antes (ej. la constante pasó de std=0.51 a 0.69).

Pero siguen siendo muy significativos (p<0.001) → conclusión: la relación inflación → yield sigue siendo estadísticamente fuerte.

Ahora los intervalos de confianza son más realistas, porque tienen en cuenta la autocorrelación.

✅ En resumen: ya tienemos un modelo lineal robusto que muestra que por cada +1% de inflación, el rendimiento a 10 años sube ~0.65% en promedio, con bastante confianza.

### Modelo VAR (Yield + Inflación)

In [ ]:

# Selección de variables
var_data = df_clean[["yield_10y", "inflation_yoy"]].dropna()

# Crear el modelo VAR
model_var = VAR(var_data)

# Selección de número óptimo de rezagos (lags)
lag_order = model_var.select_order(maxlags=8)
print("\n--- Selección de rezagos óptimos ---")
print(lag_order.summary())

# Ajustar el VAR con el lag óptimo (ejemplo: AIC)
best_lag = lag_order.aic
results_var = model_var.fit(best_lag)

print("\n--- Resumen VAR ---")
print(results_var.summary())

# Pronóstico 5 años adelante
forecast = results_var.forecast(var_data.values[-best_lag:], steps=5)
forecast_df = pd.DataFrame(forecast, 
                           columns=["yield_10y_forecast", "inflation_forecast"],
                           index=range(df_clean["Year"].max()+1, df_clean["Year"].max()+6))

print("\n--- Pronóstico 5 años ---")
print(forecast_df)


#### 1. Selección de rezagos

AIC → lag 3 (destacado con *).

Esto significa que tanto yield como inflación se explican mejor mirando los últimos 3 años de historia.

Tiene sentido: los efectos de la inflación en los bonos no son inmediatos, tardan algunos años en reflejarse.

#### 2. Resultados del VAR

En el resumen VAR, cada ecuación muestra cómo una variable depende de sus propios rezagos y de los rezagos de la otra variable.

Ejemplo interpretativo (simplificado):

yield_10y depende positivamente de la inflación rezagada → confirma que inflación alta hoy tiende a subir los rendimientos en 1–3 años.

inflation_yoy también puede mostrar influencia de yield_10y pasado (aunque más débil), porque tipos de interés altos suelen enfriar la inflación.

#### 3. Pronóstico (forecast)

El modelo te dio predicciones para 5 años:

### Año	Yield 10y (%)	Inflación (%)
2024	~4.74	~6.44

2025	~5.21	~6.03

2026	~5.86	~6.39

2027	~6.43	~6.52

2028	(seguiría creciendo en línea)	

#### Interpretación rápida:
El modelo cree que si la dinámica histórica se mantiene:

Los rendimientos del 10 años subirían de ~4.7% a ~6.4% en 3 años.

La inflación se mantendría alrededor de 6% → persistente, no se reduce rápido.

#### Gráfico histórico + forecast VAR

In [ ]:

# Datos históricos
hist_years = df_clean["Year"]
hist_yield = df_clean["yield_10y"]
hist_infl = df_clean["inflation_yoy"]

# Forecast del VAR (lo que ya calculamos antes)
forecast_df.index.name = "Year"

# Unimos histórico + forecast
plt.figure(figsize=(12,6))

# Yield
plt.plot(hist_years, hist_yield, label="Yield 10y (histórico)", color="blue")
plt.plot(forecast_df.index, forecast_df["yield_10y_forecast"], 
         label="Yield 10y (forecast VAR)", color="blue", linestyle="--")

# Inflación
plt.plot(hist_years, hist_infl, label="Inflación YoY (histórico)", color="red")
plt.plot(forecast_df.index, forecast_df["inflation_forecast"], 
         label="Inflación YoY (forecast VAR)", color="red", linestyle="--")

plt.axvline(x=2024, color="gray", linestyle="--", alpha=0.6)  # separación entre histórico y forecast
plt.title("Evolución y Forecast (VAR) - Yield 10y vs Inflación")
plt.xlabel("Año")
plt.ylabel("%")
plt.legend()
plt.grid(True)
plt.show()


#### Interpretación del primer gráfico (Histórico + Forecast VAR)

Lo que vemos en azul (Yield 10 años)

La línea azul continua: es la evolución histórica del rendimiento de los bonos del Tesoro a 10 años (yield).

La línea azul discontinua: es la predicción del modelo VAR para los próximos años.

Observamos que el yield tuvo picos muy altos en los 70–80 (crisis de inflación) y luego una tendencia descendente hasta mínimos recientes.

El forecast proyecta un ligero repunte del yield, lo que indica que los tipos de interés reales podrían subir en el futuro.

Lo que vemos en rojo (Inflación YoY)

La línea roja continua: es la inflación histórica. Destacan los picos en los años 70 (shocks petroleros) y el repunte fuerte en 2021–2022.

La línea roja discontinua: es la predicción del VAR. Muestra que la inflación podría bajar desde los picos recientes, pero aún mantenerse algo elevada comparada con los 2010s.

Interpretación conjunta

El modelo VAR nos dice que yield e inflación están claramente relacionadas en el tiempo: cuando sube la inflación, el yield tiende a subir después (los inversores exigen más rentabilidad para compensar la pérdida de poder adquisitivo).

La proyección indica un escenario de inflación todavía algo elevada con yields acompañando al alza → típico de un contexto post-crisis inflacionaria.

#### En resumen:

El gráfico confirma la relación positiva entre inflación y yield.

El modelo espera que en los próximos años ambos suban moderadamente, no a niveles extremos como los 70, pero tampoco tan bajos como en los 2010s.

###  VAR completo: selección de rezagos + ajuste + forecast + IRF
 Requiere: statsmodels, pandas, matplotlib
 Usa df_clean con columnas: Year, yield_10y, inflation_yoy

In [ ]:

# 0) Preparar datos para VAR
df_var = df_clean[["yield_10y", "inflation_yoy"]].dropna().copy()

# 1) Selección de rezagos óptimos
model_var = VAR(df_var)
sel = model_var.select_order(maxlags=8)
print("\n--- Selección de rezagos ---")
print(sel.summary())

# Tomamos el lag con mejor AIC (puedes cambiar a .bic/.hqic/.fpe si prefieres)
best_lag = sel.aic
print(f"\nLag elegido (AIC): {best_lag}")

# 2) Ajuste del VAR con ese lag
var_res = model_var.fit(best_lag)
print("\n--- Resumen VAR ---")
print(var_res.summary())

# 3) Forecast 5 pasos hacia adelante
steps = 5
last_year = int(df_clean["Year"].max())
fcast = var_res.forecast(df_var.values[-best_lag:], steps=steps)
forecast_df = pd.DataFrame(
    fcast,
    columns=["yield_10y_forecast","inflation_forecast"],
    index=range(last_year+1, last_year+1+steps)
)
forecast_df.index.name = "Year"
print("\n--- Forecast ---")
print(forecast_df)

# 4) Gráfico histórico + forecast (opcional si ya lo tenías)
plt.figure(figsize=(12,6))
plt.plot(df_clean["Year"], df_clean["yield_10y"], label="Yield 10y (histórico)")
plt.plot(forecast_df.index, forecast_df["yield_10y_forecast"], "--", label="Yield 10y (forecast VAR)")
plt.plot(df_clean["Year"], df_clean["inflation_yoy"], label="Inflación YoY (histórico)")
plt.plot(forecast_df.index, forecast_df["inflation_forecast"], "--", label="Inflación YoY (forecast VAR)")
plt.axvline(x=last_year, color="gray", linestyle="--", alpha=0.6)
plt.title("Evolución y Forecast (VAR) - Yield 10y vs Inflación")
plt.xlabel("Año"); plt.ylabel("%"); plt.legend(); plt.grid(True); plt.tight_layout(); plt.show()

# 5) IRF (Impulse Response Functions)
irf_h = 10  # horizonte en años
irf = var_res.irf(irf_h)

# IRF combinadas
fig = irf.plot(orth=True)
plt.suptitle("Funciones de Impulso-Respuesta (IRF)", fontsize=14)
plt.show()

# IRF específicas (opcional): respuesta de yield a shock de inflación, y viceversa
fig = irf.plot_cum_effects(orth=True)
plt.suptitle("IRF acumuladas (orth)", fontsize=14)
plt.show()


#### Los resultados del modelo VAR muestran que:

La inflación impulsa al rendimiento del bono a 10 años, confirmando que mayor inflación esperada eleva los tipos largos.

El efecto contrario (yields sobre inflación) existe pero es más débil y menos persistente.

Tanto la inflación como los yields presentan persistencia en sus shocks, manteniendo efectos durante varios periodos.

El pronóstico VAR anticipa un repunte moderado en ambas variables, con algo más de volatilidad en la inflación.

### Análisis VAR: Diagnóstico y Extensiones

In [ ]:
# 1) Ljung–Box por variable (univariante)
for col in var_res.resid.columns:
    print(f"\nLjung–Box (lags=10) para {col}")
    print(acorr_ljungbox(var_res.resid[col].dropna(), lags=[10], return_df=True))


In [ ]:
# 2) Durbin–Watson (otra medida de autocorrelación)
dw = durbin_watson(var_res.resid.values)
for col, val in zip(var_res.resid.columns, dw):
    print(f"Durbin–Watson {col}: {val:.2f}")  # ~2 es bueno (sin autocorrelación)


In [ ]:
# 3) Estabilidad del VAR
var_res.is_stable(verbose=True)  # Debe devolver True / raíces dentro del círculo unitario


In [ ]:
# 4) Heterocedasticidad (ARCH) por variable
for col in var_res.resid.columns:
    stat, pval, _, _ = het_arch(var_res.resid[col].dropna())
    print(f"ARCH para {col}: p-value = {pval:.4f}")  # >= 0.05 ⇒ OK


In [ ]:
# 5) Normalidad conjunta de residuos
norm = var_res.test_normality()
print(norm.summary())  # p-value alto ⇒ OK


In [ ]:
for col in var_res.resid.columns:
    plt.figure(); plot_acf(var_res.resid[col].dropna(), lags=20); plt.title(f"ACF {col}")
    plt.figure(); plot_pacf(var_res.resid[col].dropna(), lags=20); plt.title(f"PACF {col}")
    plt.show()


#### Interpretación de los resultados (Punto 3.1)

En los gráficos de ACF y PACF:

La mayoría de las barras caen dentro de la banda azul (intervalo de confianza).

Eso significa que no hay autocorrelación significativa en los residuos.

Un modelo VAR bien especificado debe dejar los residuos como “ruido blanco”, y eso es lo que se ve aquí.

## 3.2 Test de autocorrelación alternativa (Durbin–Watson)

El estadístico Durbin–Watson verifica la autocorrelación de primer orden.  
- Valor cercano a **2** ⇒ sin autocorrelación.  
- Valor < 2 ⇒ autocorrelación positiva.  
- Valor > 2 ⇒ autocorrelación negativa.


In [ ]:
dw = durbin_watson(var_res.resid.values)
for col, val in zip(var_res.resid.columns, dw):
    print(f"Durbin–Watson {col}: {val:.2f}")


## 3.3 Test de estabilidad del VAR

Un VAR estable tiene todas sus raíces dentro del círculo unitario.  
Si es **True**, los pronósticos son fiables.


In [ ]:
var_res.is_stable(verbose=True)


## 3.4 Test de heterocedasticidad (ARCH)

El test ARCH evalúa si la varianza de los residuos es constante.  
- **p-value ≥ 0.05** ⇒ no hay heterocedasticidad (bien).  
- **p-value < 0.05** ⇒ problemas de heterocedasticidad.


In [ ]:
for col in var_res.resid.columns:
    stat, pval, _, _ = het_arch(var_res.resid[col].dropna())
    print(f"ARCH {col}: p-value = {pval:.4f}")


## 3.5 Test de normalidad de residuos

El test de Jarque–Bera verifica si los residuos siguen una distribución normal.  
- **p-value ≥ 0.05** ⇒ no rechazamos normalidad (OK).  
- **p-value < 0.05** ⇒ residuos no normales.


In [ ]:
norm = var_res.test_normality()
print(norm.summary())


## 4. Pronóstico extendido con intervalos de confianza

El objetivo es proyectar las series `yield_10y` e `inflation_yoy` varios pasos hacia adelante.
Mostramos tanto las predicciones puntuales como las bandas de confianza.


In [ ]:
# ==== 4. Pronóstico extendido con intervalos de confianza (VARResults) ====
steps = 10  # años a proyectar
last_year = int(df_clean["Year"].max())

# nº de rezagos que usó el VAR
k = var_res.k_ar

# Pronóstico: medias y bandas (lower/upper)
fcast_mean, fcast_lower, fcast_upper = var_res.forecast_interval(
    y=var_res.endog[-k:],  # las últimas k observaciones como estado inicial
    steps=steps,
    alpha=0.05             # 95% IC
)

cols = var_res.names  # ['yield_10y','inflation_yoy']

# DataFrames ordenados con índice de años futuros
idx_future = range(last_year+1, last_year+steps+1)
f_mean  = pd.DataFrame(fcast_mean,  index=idx_future, columns=cols)
f_lower = pd.DataFrame(fcast_lower, index=idx_future, columns=cols)
f_upper = pd.DataFrame(fcast_upper, index=idx_future, columns=cols)

print("\n--- Pronóstico extendido (medias) ---")
print(f_mean)


### Gráfico histórico + forecast con bandas

In [ ]:
plt.figure(figsize=(12,6))

for col in cols:
    # histórico
    plt.plot(df_clean["Year"], df_clean[col], label=f"{col} (histórico)")
    # predicción
    plt.plot(f_mean.index, f_mean[col], linestyle="--", label=f"{col} (forecast)")
    # intervalos
    plt.fill_between(f_mean.index, f_lower[col], f_upper[col], alpha=0.2)

plt.axvline(x=last_year, color="gray", linestyle="--", alpha=0.7)
plt.title("Pronóstico extendido VAR con intervalos de confianza (95%)")
plt.xlabel("Año"); plt.ylabel("%"); plt.legend(); plt.grid(True); plt.show()


### Interpretación rápida:

Se observa que el yield_10y tendería a estabilizarse, mientras que la inflación muestra un ligero descenso en el forecast, aunque con bastante incertidumbre (bandas amplias).

Esto refleja la lógica: el modelo VAR capta relaciones, pero a largo plazo las predicciones son menos seguras.

## 5. Descomposición de varianza del error de pronóstico (FEVD)

Mide qué porcentaje del error de predicción de cada variable se explica por shocks propios y por la otra variable, a distintos horizontes.


In [ ]:
steps = 10  # horizonte FEVD
fevd = var_res.fevd(steps)

# Resumen en texto
print(fevd.summary())

# Gráfico por variable
_ = fevd.plot(figsize=(10,6))


## 6. Causalidad de Granger

Contrasta si los rezagos de una variable ayudan a predecir a la otra (más allá de sus propios rezagos).
- p-value < 0.05 ⇒ Rechazamos “no causa” ⇒ hay causalidad de Granger en esa dirección.


In [ ]:
# ¿Inflation_yoy causa (Granger) a yield_10y?
print(var_res.test_causality('yield_10y', ['inflation_yoy'], kind='f').summary())

# ¿Yield_10y causa (Granger) a inflation_yoy?
print(var_res.test_causality('inflation_yoy', ['yield_10y'], kind='f').summary())


### 7. Funciones de Impulso-Respuesta (IRF)

#### 7.1 IRF ortogonalizadas (Cholesky) con bandas de confianza

In [ ]:
# Horizonte (años)
h = 10

# IRF básicas
irf = var_res.irf(h)

# Gráfico de IRF ortogonalizadas (Cholesky)
fig = irf.plot(orth=True)
fig.suptitle("IRF ortogonalizadas (Cholesky)", fontsize=14)

# Bandas de confianza por bootstrap Monte Carlo
# (repl=1000 si quieres más precisión; tardará más)
fig_ci = irf.errband_mc(orth=True, repl=500)  # devuelve fig y ejes con bandas


#### 7.2 IRF acumuladas (efecto total a lo largo del horizonte)

In [ ]:
fig_cum = irf.plot_cum_effects(orth=True)
fig_cum.suptitle("IRF acumuladas (orth)", fontsize=14)


#### 7.3 Sensibilidad al orden de las variables
Cambiar el orden es buena práctica de robustez: primero inflación, luego yield.

In [ ]:
# Re-ajustamos el VAR con columnas reordenadas
df_alt = df_clean[["inflation_yoy", "yield_10y"]].dropna().copy()
model_alt = VAR(df_alt)
var_res_alt = model_alt.fit(var_res.k_ar)  # usa el mismo nº de rezagos

irf_alt = var_res_alt.irf(h)
fig_alt = irf_alt.plot(orth=True)
fig_alt.suptitle("IRF orth con orden alternativo (inflación primero)", fontsize=14)


#### 7.4 Guardar figuras a disco

In [ ]:
fig.savefig("irf_orth.png", dpi=150, bbox_inches="tight")
fig_cum.savefig("irf_cum_orth.png", dpi=150, bbox_inches="tight")
fig_alt.savefig("irf_orth_alt_order.png", dpi=150, bbox_inches="tight")


###  Interpretación Punto 7 (IRFs)

Shock inflación → Yield 10Y: impacto fuerte y sostenido, los bonos suben porque el mercado exige más rentabilidad.

Shock Yield 10Y → Inflación: efecto débil y pasajero, incluso negativo (tipos altos enfrían la economía).

Inflación sobre sí misma: persistente, pero va perdiendo fuerza.

Yield sobre sí mismo: se corrige rápido.

IRFs acumuladas: confirman que la inflación arrastra a los yields, no al revés.

Sensibilidad al orden: los resultados son robustos, no dependen del orden de variables.

📌 Conclusión clara:
La inflación lidera y los yields siguen.

#### Preparar el dataset USA para el pipeline


In [ ]:
df_usa = (
    df_clean[['Year', 'yield_10y', 'inflation_yoy']]
      .dropna()
      .copy()
)

df_usa['Year'] = df_usa['Year'].astype(int)
df_usa = df_usa.set_index('Year').sort_index()

#### Define el pipeline (elige rezagos automáticamente)

In [ ]:
def run_country_pipeline_auto(df, country_name, steps=10, maxlags=8, crit="aic"):
    """
    Pipeline VAR completo con selección automática de rezagos.
    df: DataFrame con index=Year y columnas ['yield_10y','inflation_yoy']
    steps: horizonte de forecast
    maxlags: rezago máximo a evaluar
    crit: 'aic' | 'bic' | 'hqic' | 'fpe'
    """
    df = df[['yield_10y','inflation_yoy']].dropna().copy()
    model = VAR(df)

    # 1) Selección de rezagos
    sel = model.select_order(maxlags=maxlags)
    best_lag = getattr(sel, crit)
    print(f"\n[{country_name}] Rezagos óptimos por {crit.upper()}: {best_lag}")

    # 2) Ajuste
    res = model.fit(best_lag)
    print(res.summary())

    # 3) Diagnóstico (rápido)
    print("\n--- Diagnóstico ---")
    for col in res.resid.columns:
        lb = acorr_ljungbox(res.resid[col].dropna(), lags=[10], return_df=True)
        print(f"Ljung-Box {col} (lag=10): p-value={lb['lb_pvalue'].iloc[0]:.4f}")
    dw = durbin_watson(res.resid.values)
    for c,v in zip(res.resid.columns, dw):
        print(f"Durbin–Watson {c}: {v:.2f}")
    print("Estabilidad:", res.is_stable(verbose=True))

    # 4) Forecast con bandas
    k = res.k_ar
    mean, low, up = res.forecast_interval(res.endog[-k:], steps=steps, alpha=0.05)
    idx_future = range(df.index.max()+1, df.index.max()+1+steps)
    f_mean = pd.DataFrame(mean, index=idx_future, columns=res.names)
    f_low  = pd.DataFrame(low,  index=idx_future, columns=res.names)
    f_up   = pd.DataFrame(up,   index=idx_future, columns=res.names)

    plt.figure(figsize=(11,5))
    for col in res.names:
        plt.plot(df.index, df[col], label=f"{col} (hist.)")
        plt.plot(f_mean.index, f_mean[col], "--", label=f"{col} (fcst)")
        plt.fill_between(f_mean.index, f_low[col], f_up[col], alpha=0.2)
    plt.axvline(df.index.max(), color="gray", linestyle="--", alpha=0.7)
    plt.title(f"{country_name} – Forecast VAR (95% IC)")
    plt.xlabel("Año"); plt.ylabel("%"); plt.legend(); plt.grid(True); plt.show()

    # 5) FEVD
    fevd = res.fevd(steps)
    print("\n--- FEVD ---")
    print(fevd.summary())
    fevd.plot(figsize=(9,5)); plt.show()

    # 6) Granger
    print("\n--- Granger ---")
    print(res.test_causality('yield_10y',['inflation_yoy'], kind='f').summary())
    print(res.test_causality('inflation_yoy',['yield_10y'], kind='f').summary())

    # 7) IRFs (orth y acumuladas)
    irf = res.irf(steps)
    irf.plot(orth=True); plt.suptitle(f"{country_name} – IRF orth"); plt.show()
    irf.plot_cum_effects(orth=True); plt.suptitle(f"{country_name} – IRF acumuladas"); plt.show()

    return res, {"forecast_mean": f_mean, "forecast_low": f_low, "forecast_up": f_up}, fevd, irf, best_lag


#### Ejecutar el pipeline para USA

In [ ]:
usa_results, usa_fcst, usa_fevd, usa_irf, usa_bestlag = run_country_pipeline_auto(
    df_usa, country_name="USA", steps=10, maxlags=8, crit="aic"
)
print("Lag elegido (AIC):", usa_bestlag)


### Detectar la columna 10Y de Alemania en yields.csv

In [ ]:
# Cargar si no esta ya en memoria:
yields = pd.read_csv(YIELDS_CSV)

# columnas 10Y tipo 'US10', 'DE10', etc.
ten_cols = [c for c in yields.columns if re.fullmatch(r"[A-Z]{2}\d{2}", c) and c.endswith("10")]
print("10Y detectadas (primeras 30):", ten_cols[:30])

# Intento automático para Alemania
candidatos_de = [c for c in ten_cols if c.startswith(("DE","GE","BD","GM"))]
print("Candidatos Alemania:", candidatos_de)

# Si aparece 'DE10', úsala; si no, toma el primer candidato que salga.
col_de = "DE10" if "DE10" in ten_cols else (candidatos_de[0] if candidatos_de else None)
print("Columna usada para Alemania 10Y:", col_de)


### Anualizar yields USA + Alemania

In [ ]:
yields_yr = yields.copy()
yields_yr['Year'] = pd.to_datetime(yields_yr['time'], unit='ms').dt.year

col_us = "US10"   # USA 10Y
col_de = "DE10"   # Alemania 10Y

y_ann = (
    yields_yr[['Year', col_us, col_de]]
      .groupby('Year', as_index=False)
      .mean()
      .rename(columns={col_us:'yield_10y_US', col_de:'yield_10y_DE'})
)


### Inflación del Banco Mundial (USA + Alemania)

In [ ]:
wb = pd.read_csv(WORLD_BANK_DATA)

wb_small = wb.rename(columns={
    'country_name':'Country',
    'year':'Year',
    'Inflation (CPI %)':'inflation_yoy'
})[['Country','Year','inflation_yoy']].dropna()

infl_us = wb_small.query("Country == 'United States'")[['Year','inflation_yoy']].rename(columns={'inflation_yoy':'inflation_yoy_US'})
infl_de = wb_small.query("Country == 'Germany'")[['Year','inflation_yoy']].rename(columns={'inflation_yoy':'inflation_yoy_DE'})


### Construir datasets y lanzar el pipeline

#### Carga el CSV de Alemania

In [ ]:
# === 1) Cargar CSV Alemania desde FRED ===
de_raw = pd.read_csv(IRLTLT)

# Normalizar columnas
de_raw['observation_date'] = pd.to_datetime(de_raw['observation_date'], errors='coerce')
de_raw = de_raw.rename(columns={'IRLTLT01DEM156N':'yield_10y'})

# === 2) Pasar de mensual → anual (promedio) ===
de_annual = de_raw.set_index('observation_date').resample('YE').mean()
de_annual.index = de_annual.index.year
de_annual.index.name = 'Year'
de_annual = de_annual.reset_index()

print("Alemania yields anuales:", de_annual['Year'].min(), "→", de_annual['Year'].max())
display(de_annual.head())

# === 3) Inflación Alemania desde World Bank ===
wb_small = wb.rename(columns={
    'country_name':'Country',
    'year':'Year',
    'Inflation (CPI %)':'inflation_yoy'
})[['Country','Year','inflation_yoy']].dropna()

infl_de = wb_small.query("Country == 'Germany'")[['Year','inflation_yoy']]

# === 4) Merge yields + inflación ===
df_germany = (
    de_annual.merge(infl_de, on='Year', how='inner')
             .set_index('Year').sort_index()
)

print("Alemania combinado:", df_germany.index.min(), "→", df_germany.index.max())
display(df_germany.tail())


### Alemania: cortar a 2010–2024 y correr el

In [ ]:
# --- Alemania 2010–2024 ---
df_germany_2010_24 = df_germany.loc[2010:2024].copy()

N = len(df_germany_2010_24)
safe_maxlags = max(1, min(3, (N - 3)//2))   # con ~15 años suele salir 1–2
print(f"[Germany] N={N} | maxlags sugerido={safe_maxlags}")

de_res, de_fcst, de_fevd, de_irf, de_bestlag = run_country_pipeline_auto(
    df_germany_2010_24, country_name="Germany (2010–2024)",
    steps=10, maxlags=safe_maxlags, crit="aic"
)
print("Germany (2010–2024) – lag elegido:", de_bestlag)


### USA: mismo corte 2010–2024

In [ ]:
def run_country_pipeline_auto(df, country_name, steps=5, maxlags=2, crit="aic"):
    """
    df: index=Year (int), columnas ['yield_10y','inflation_yoy']
    steps: horizonte de forecast
    maxlags: rezago máximo permitido (luego se capea automáticamente)
    """
    # 0) Formato y limpieza
    df = df[['yield_10y','inflation_yoy']].dropna().copy()
    df.index = df.index.astype(int)
    df = df.sort_index()

    # 1) Cálculo de un maxlags seguro según nº de observaciones
    N = len(df)
    k_endog = df.shape[1]  # 2
    safe_max = max(1, min(maxlags, (N - 5)//k_endog))  # cap defensivo
    if safe_max < 1:
        raise ValueError(f"Datos insuficientes (N={N}). Reduce rango de años o añade datos.")
    if safe_max < maxlags:
        print(f"[{country_name}] maxlags reducido de {maxlags} a {safe_max} por N={N}")

    # 2) Selección de rezagos y ajuste
    model = VAR(df)
    sel = model.select_order(maxlags=safe_max)
    best_lag = getattr(sel, crit)
    if best_lag is None or best_lag < 1:
        best_lag = min(1, safe_max)
    print(f"\n[{country_name}] Rezagos óptimos por {crit.upper()}: {best_lag}")
    res = model.fit(best_lag)
    print(res.summary())

    # 3) Diagnóstico robusto (Ljung–Box con lag seguro)
    print("\n--- Diagnóstico ---")
    nres = len(res.resid)
    lb_lag = max(1, min(10, nres - 2, 2*best_lag))  # evita el error por tamaños
    for col in res.resid.columns:
        if lb_lag >= 1:
            lb = acorr_ljungbox(res.resid[col].dropna(), lags=[lb_lag], return_df=True)
            print(f"Ljung-Box {col} (lag={lb_lag}): p={lb['lb_pvalue'].iloc[0]:.4f}")
        else:
            print(f"Ljung-Box {col}: saltado (muy pocas observaciones)")
    dw = durbin_watson(res.resid.values)
    for c, v in zip(res.resid.columns, dw):
        print(f"Durbin–Watson {c}: {v:.2f}")
    print("Estabilidad:", res.is_stable(verbose=True))

    # 4) Forecast con bandas (95%)
    k = res.k_ar
    mean, low, up = res.forecast_interval(res.endog[-k:], steps=steps, alpha=0.05)
    idx_future = range(df.index.max()+1, df.index.max()+1+steps)
    f_mean = pd.DataFrame(mean, index=idx_future, columns=res.names)
    f_low  = pd.DataFrame(low,  index=idx_future, columns=res.names)
    f_up   = pd.DataFrame(up,   index=idx_future, columns=res.names)

    plt.figure(figsize=(11,5))
    for col in res.names:
        plt.plot(df.index, df[col], label=f"{col} (hist.)")
        plt.plot(f_mean.index, f_mean[col], "--", label=f"{col} (fcst)")
        plt.fill_between(f_mean.index, f_low[col], f_up[col], alpha=0.2)
    plt.axvline(df.index.max(), color="gray", linestyle="--", alpha=0.7)
    plt.title(f"{country_name} – Forecast VAR (95% IC)")
    plt.xlabel("Año"); plt.ylabel("%"); plt.legend(); plt.grid(True); plt.show()

    # 5) FEVD
    fevd = res.fevd(steps)
    print("\n--- FEVD ---")
    print(fevd.summary())
    fevd.plot(figsize=(9,5)); plt.show()

    # 6) Causalidad de Granger
    print("\n--- Granger ---")
    print(res.test_causality('yield_10y', ['inflation_yoy'], kind='f').summary())
    print(res.test_causality('inflation_yoy', ['yield_10y'], kind='f').summary())

    # 7) IRFs (orth y acumuladas)
    irf = res.irf(steps)
    irf.plot(orth=True); plt.suptitle(f"{country_name} – IRF orth"); plt.show()
    irf.plot_cum_effects(orth=True); plt.suptitle(f"{country_name} – IRF acumuladas"); plt.show()

    return res, {"forecast_mean": f_mean, "forecast_low": f_low, "forecast_up": f_up}, fevd, irf, best_lag


In [ ]:
df_usa_2010_24     = df_usa.loc[2010:2024].copy()
df_germany_2010_24 = df_germany.loc[2010:2024].copy()

print("USA 2010–2024:", df_usa_2010_24.index.min(), "→", df_usa_2010_24.index.max(), "| N=", len(df_usa_2010_24))
print("DE  2010–2024:", df_germany_2010_24.index.min(), "→", df_germany_2010_24.index.max(), "| N=", len(df_germany_2010_24))


In [ ]:
# USA
us_res, us_fcst, us_fevd, us_irf, us_bestlag = run_country_pipeline_auto(
    df_usa_2010_24, country_name="USA (2010–2024)", steps=5, maxlags=2, crit="aic"
)
print("USA (2010–2024) – lag elegido:", us_bestlag)




In [ ]:
assert {'yield_10y','inflation_yoy'}.issubset(df_usa.columns), "df_usa no tiene las columnas correctas"
df_usa_2010_24 = df_usa.loc[2010:2024].copy()
print("Rango USA recortado:", df_usa_2010_24.index.min(), "→", df_usa_2010_24.index.max(), "| N=", len(df_usa_2010_24))
print(df_usa_2010_24.head(), "\n", df_usa_2010_24.tail())


In [ ]:
# Años disponibles en cada fuente (USA)
print("Yields USA (y_ann):", y_ann[['Year','yield_10y_US']].dropna()['Year'].min(), "→", y_ann[['Year','yield_10y_US']].dropna()['Year'].max())
print("Inflación USA (infl_us):", infl_us['Year'].min(), "→", infl_us['Year'].max())

# Qué años faltan en cada una dentro de 2010–2024
yrs = set(range(2010, 2025))
y_ok  = set(y_ann.loc[y_ann['yield_10y_US'].notna(), 'Year'])
i_ok  = set(infl_us['Year'])
print("Faltan en yields:", sorted(yrs - y_ok))
print("Faltan en inflación:", sorted(yrs - i_ok))


#### Anualizar DGS10 y crear Year

In [ ]:
# Cargar CSV DGS10 (tiene columnas: observation_date, DGS10)
dgs = pd.read_csv(DGS10)

# Limpiar tipos
dgs["observation_date"] = pd.to_datetime(dgs["observation_date"], errors="coerce")
dgs["DGS10"] = pd.to_numeric(dgs["DGS10"], errors="coerce")
dgs = dgs.dropna(subset=["observation_date", "DGS10"]).sort_values("observation_date")

# Resample anual (promedio) y crear Year de forma explícita
dgs_ann = (
    dgs.set_index("observation_date")
       .resample("YE").mean()                  # promedio anual
       .assign(Year=lambda x: x.index.year)    # crear Year desde el índice
       .reset_index(drop=True)[["Year", "DGS10"]]
       .rename(columns={"DGS10": "yield_10y"})
)

# Filtrar 2021–2024 (o lo que tengas)
dgs_ann = dgs_ann.query("Year >= 2021").copy()

print(dgs_ann.head())


#### Unir con tu serie USA previa

In [ ]:
# Recalcular US10 anual (hasta 2020) desde tu yields.csv
yields_yr = yields.copy()
yields_yr['Year'] = pd.to_datetime(yields_yr['time'], unit='ms').dt.year

us10_ann_old = (
    yields_yr.groupby('Year')['US10'].mean()
             .rename('yield_10y')
             .reset_index()
)

print("US10 anualizado (hasta 2020):")
print(us10_ann_old.tail())


In [ ]:
# us10_ann_old: tu serie histórica hasta 2020 (Year, yield_10y)
us10_full = (
    pd.concat([us10_ann_old, dgs_ann], ignore_index=True)
      .sort_values("Year")
      .drop_duplicates("Year", keep="last")
      .reset_index(drop=True)
)

print(us10_full.tail(15))


### Construir df_usa_2010_24 para el VAR

In [ ]:
# Asegúrate de tener infl_us: (Year, inflation_yoy)
df_usa_2010_24 = (
    us10_full.merge(infl_us, on="Year", how="inner")
             .query("Year >= 2010 and Year <= 2024")
             .set_index("Year")
             .sort_index()
             .dropna()
)

print(df_usa_2010_24.tail(10))


### USA (2015–2024) – VAR robusto con cap automático de lags

In [ ]:
# Usa tu DataFrame ya preparado (índice Year, cols: yield_10y, inflation_yoy[_US])
df_us = df_usa_2015_24.copy() if 'df_usa_2015_24' in globals() else df_usa_2010_24.loc[2015:2024].copy()
if 'inflation_yoy_US' in df_us.columns:
    df_us = df_us.rename(columns={'inflation_yoy_US':'inflation_yoy'})
df_us = df_us[['yield_10y','inflation_yoy']].dropna().sort_index()
df_us.index = df_us.index.astype(int)

N, k = len(df_us), 2
print(f"[USA 2015–2024] N={N}")

# --- Selección de rezagos robusta ---
# Cap muy prudente con muestras cortas:
max_try = max(1, min(2, (N-5)//k))  # con N~10 esto da 1–2
best_lag = None
last_err = None

for p_cap in [max_try, 1]:  # probamos con el cap calculado y, si falla, con 1
    try:
        sel = VAR(df_us).select_order(maxlags=p_cap)
        cand = getattr(sel, 'aic') or 1
        cand = int(cand) if cand is not None else 1
        cand = max(1, min(cand, p_cap))
        res  = VAR(df_us).fit(cand)
        best_lag = cand
        break
    except Exception as e:
        last_err = e
        continue

if best_lag is None:
    raise last_err

print(f"Rezago óptimo (AIC, cap={p_cap}): {best_lag}")
print(res.summary())

# --- Diagnóstico robusto ---
lb_lag = max(1, min(5, N - 2, 2*best_lag))
print("\n--- Diagnóstico ---")
for col in res.resid.columns:
    lb = acorr_ljungbox(res.resid[col].dropna(), lags=[lb_lag], return_df=True)
    print(f"Ljung-Box {col} (lag={lb_lag})  p={lb['lb_pvalue'].iloc[0]:.3f}")
dw_vals = durbin_watson(res.resid.values)
print("Durbin–Watson:", {c: round(dw,2) for c, dw in zip(res.resid.columns, dw_vals)})
print("Estabilidad:", res.is_stable(verbose=True))

# --- Forecast (5 años, sin IC para N pequeño) ---
steps = 5
yhat = res.forecast(df_us.values[-best_lag:], steps=steps)
years_fc = np.arange(df_us.index.max()+1, df_us.index.max()+steps+1)
fc_df = pd.DataFrame(yhat, columns=df_us.columns, index=years_fc)

plt.figure(figsize=(11,6))
plt.plot(df_us.index, df_us['yield_10y'], label='yield_10y (hist)')
plt.plot(df_us.index, df_us['inflation_yoy'], label='inflation_yoy (hist)')
plt.plot(fc_df.index, fc_df['yield_10y'], '--', label='yield_10y (fcst)')
plt.plot(fc_df.index, fc_df['inflation_yoy'], '--', label='inflation_yoy (fcst)')
plt.axvline(x=df_us.index.max(), color='grey', ls='--', alpha=0.6)
plt.title('USA (2015–2024) – VAR forecast (5 años)')
plt.xlabel('Año'); plt.ylabel('%'); plt.grid(True); plt.legend(); plt.tight_layout(); plt.show()

# --- FEVD e IRFs (10 pasos) ---
fevd = res.fevd(10); fig = fevd.plot(figsize=(10,6)); fig.suptitle('USA – FEVD (10)'); plt.show()
irf = res.irf(10); fig1 = irf.plot(orth=True); fig1.suptitle('USA – IRF (orth)'); plt.show()
fig2 = irf.plot_cum_effects(orth=True); fig2.suptitle('USA – IRF acumuladas (orth)'); plt.show()


## Comparativa Alemania vs USA (2010–2024)

#### 🇩🇪 Alemania

Yields 10Y: en mínimos históricos, incluso negativos 2016–2020.

Inflación: fuerte repunte desde 2021 por crisis energética y guerra en Ucrania.

Relación: shocks de inflación → suben los yields (mercado exige más rentabilidad).

#### 🇺🇸 Estados Unidos

Yields 10Y: bajan fuerte en 2020 (COVID), suben rápido hasta 2024.

Inflación: pico histórico en 2022 (8%) tras pandemia y energía.

Relación: inflación explica gran parte de la variabilidad de yields; modelo algo inestable por pocos datos.

#### 🔎 Conclusión

Inflación lidera, yields siguen en ambos países.

Alemania vivió rendimientos negativos (BCE muy expansiva), USA nunca bajó de 0.8% (Fed menos agresiva).

Dos respuestas distintas a choques globales, pero misma dirección: más inflación = yields más altos.

### SPAIN (2010–2024) – Pipeline


In [ ]:
# ---------- 0) Cargar fuentes si faltan ----------
if 'yields' not in globals():
    yields = pd.read_csv(YIELDS_CSV)
if 'wb' not in globals():
    wb = pd.read_csv(WORLD_BANK_DATA)

# ---------- 1) Yields 10Y España (ES10) → anual ----------
yields_yr = yields.copy()
# 'time' suele ser epoch ms en tu fichero
yields_yr['Year'] = pd.to_datetime(yields_yr['time'], unit='ms').dt.year

if 'ES10' not in yields_yr.columns:
    raise RuntimeError("No encuentro la columna ES10 en yields.csv")

es10_ann = (yields_yr.groupby('Year')['ES10'].mean()
                        .rename('yield_10y')
                        .reset_index())

# ---------- 2) Inflación España desde World Bank (detección robusta) ----------
cols = {c.lower(): c for c in wb.columns}
country_col = cols.get('country_name') or cols.get('country') or cols.get('country name')
year_col    = cols.get('year') or cols.get('date')

if not country_col or not year_col:
    raise RuntimeError("No encuentro columnas de país/año en el archivo WB. Revisa nombres.")

inflation_candidates = [c for c in wb.columns if "inflation" in c.lower() and "cpi" in c.lower()]
if not inflation_candidates:
    raise RuntimeError("No encuentro columna de inflación CPI en WB (busqué 'inflation' y 'CPI').")
infl_col = inflation_candidates[0]

infl_es = (wb[[country_col, year_col, infl_col]]
             .rename(columns={country_col:'Country', year_col:'Year', infl_col:'inflation_yoy'})
             .query("Country == 'Spain'")
             [['Year','inflation_yoy']]
             .dropna())

# ---------- 3) Merge + recorte 2010–2024 ----------
df_spain = (es10_ann.merge(infl_es, on='Year', how='inner')
                    .set_index('Year').sort_index())
df_spain_2010_24 = df_spain.loc[2010:2024].dropna().copy()

print("Spain – rango disponible tras merge:",
      df_spain_2010_24.index.min(), "→", df_spain_2010_24.index.max(),
      "| N =", len(df_spain_2010_24))
display(df_spain_2010_24.tail())

# ---------- 4) Ejecutar VAR (parámetros prudentes para N~10–15) ----------
sp_res, sp_fcst, sp_fevd, sp_irf, sp_bestlag = run_country_pipeline_auto(
    df_spain_2010_24, country_name="Spain (2010–2024)", steps=5, maxlags=2, crit="aic"
)
print("Spain (2010–2024) – rezagos elegidos (AIC):", sp_bestlag)


yield_10y → inflation_yoy (izquierda):

Un shock positivo en los rendimientos de 10 años genera un aumento inicial de la inflación, que se estabiliza después de 3 años.

La respuesta es moderada y se disipa rápido.

Eso sugiere que los tipos largos tienen poca capacidad de traspaso directo a la inflación en España en este periodo.

inflation_yoy → inflation_yoy (derecha):

Un shock de inflación tiende a persistir 2–3 años, con efecto negativo moderado y luego recuperación.

En otras palabras, los shocks inflacionarios en España no son totalmente transitorios, pero tampoco se vuelven explosivos.

#### 📌 Conclusión rápida (España 2010–2024):

Los tipos largos no causan fuertemente la inflación (apoyo a lo visto en Granger).

La inflación sí muestra cierta dinámica propia (persistencia).

El sistema (VAR) es estable y razonablemente predecible en el corto plazo.

## Reino Unido 🇬🇧

#### 1. Verificar columnas de yields

In [ ]:
print(yields.columns.tolist())
print(yields.head())

#### Si el índice ya es el año

In [ ]:
yields = yields.reset_index().rename(columns={'index':'Year'})


In [ ]:
# === Reino Unido ===

# 1) Rendimientos 10Y (UK)
y_uk = (
    yields[['Year','GB10']]
    .rename(columns={'GB10':'yield_10y_UK'})
    .dropna()
)

# 2) Inflación UK
infl_uk = (
    wb_small.query("Country == 'United Kingdom'")
    [['Year','inflation_yoy']]
    .rename(columns={'inflation_yoy':'inflation_yoy_UK'})
)

# 3) Merge
df_uk = (
    y_uk.merge(infl_uk, on='Year', how='inner')
         .rename(columns={'yield_10y_UK':'yield_10y',
                          'inflation_yoy_UK':'inflation_yoy'})
         .set_index('Year').sort_index()
)

# Filtrar rango 2010–2024
df_uk = df_uk.loc[2010:2024]

print("UK rango disponible:", df_uk.index.min(), "→", df_uk.index.max(), "| N=", len(df_uk))
display(df_uk.tail())


### corre el VAR y genera gráficos

In [ ]:
# --- Reino Unido: VAR 2010–2024 ---
uk_res, uk_fcst, uk_fevd, uk_irf, uk_bestlag = run_country_pipeline_auto(
    df_uk, country_name="UK (2010–2024)", steps=5, maxlags=2, crit="aic"
)
print("UK – rezago elegido (AIC):", uk_bestlag)

# (opcional) guardar figuras si tu función las pinta
plt.savefig("uk_irf_orth.png", dpi=150, bbox_inches="tight")
plt.close()


#### Reino Unido – Resultados VAR (2010–2024)

Datos disponibles:

Rendimiento 10 años (yield_10y_UK) y inflación anual (inflation_yoy).

Rango: 2010–2024 → 15 observaciones.

VAR estimado:

Rezagos óptimos: 2 (AIC).

El sistema es estable (no explota).

Forecast (2025–2029):

Los rendimientos 10Y se mantienen estables en torno a 3.5–4%.

La inflación muestra tendencia moderada: repunte hacia ~3% pero con intervalos amplios (incertidumbre alta).

Descomposición de varianza (FEVD):

Los yields dependen principalmente de sí mismos (>90%).

La inflación también es mayormente explicada por sí misma (>80–90%), con poca influencia de los rendimientos.

Causalidad de Granger:

No hay evidencia de causalidad significativa (p-valores > 0.1).

Es decir: ni la inflación predice claramente los rendimientos, ni al revés.

Respuestas a impulsos (IRF):

Un shock en los rendimientos tiene efecto muy leve sobre inflación (positivo al inicio, se disipa).

Un shock en la inflación apenas impacta los rendimientos, incluso con respuestas negativas pequeñas.

#### Conclusión corta (UK):
En Reino Unido, los rendimientos a 10 años son bastante autónomos y siguen su propia dinámica. La inflación influye muy poco y no hay relación causal fuerte entre ambas variables. El modelo proyecta estabilidad en yields y una inflación moderada, aunque con mucha incertidumbre tras 2025.

## Japón 🇯🇵

In [ ]:

# 1) Rendimientos 10Y (JP)
y_jp = (
    yields[['JP10']]
    .rename(columns={'JP10':'yield_10y'})
    .dropna()
)
y_jp.index.name = "Year"

# 2) Inflación Japón
infl_jp = (
    wb_small.query("Country == 'Japan'")
    [['Year','inflation_yoy']]
    .set_index("Year")
)

# 3) Merge de las dos series
df_jp = (
    y_jp.join(infl_jp, how="inner")
    .rename(columns={'inflation_yoy':'inflation_yoy'})
)

# 4) Recorte 2010–2024
df_jp = df_jp.loc[2010:2024]

print("JP rango disponible:", df_jp.index.min(), "-", df_jp.index.max(), "| N=", len(df_jp))
display(df_jp.tail())


In [ ]:
# === JAPÓN – VAR completo y robusto (crea res_jp y hace forecast con bandas) ===
# 0) Asegurar formato
df_jp = df_jp[['yield_10y','inflation_yoy']].dropna().copy()
df_jp.index = df_jp.index.astype(int)
df_jp = df_jp.sort_index().loc[2010:2024]
N = len(df_jp)
print(f"[JP] N={N} | años {df_jp.index.min()}–{df_jp.index.max()}")

# 1) Selección de rezagos robusta (cap descendente)
model_jp = VAR(df_jp)
safe_p = None
for cap in [4,3,2,1]:
    try:
        sel = model_jp.select_order(cap)
        p = sel.aic if sel.aic is not None else 1
        p = int(max(1, min(p, cap)))
        res_jp = model_jp.fit(p)
        safe_p = p
        print(f"Rezago elegido (AIC dentro de cap={cap}): p={p}")
        break
    except Exception as e:
        print(f"cap={cap} no estimable → {e}")

if safe_p is None:
    # última red
    safe_p = 1
    res_jp = model_jp.fit(safe_p)
    print("Forzado p=1")

print(res_jp.summary())
print("Estabilidad VAR:", res_jp.is_stable())

# 2) Forecast con bandas (5 años)
steps = 5
p = res_jp.k_ar
last_Y = df_jp.values[-p:]                       # condiciones iniciales
fc_mean, fc_lo, fc_hi = res_jp.forecast_interval(last_Y, steps=steps, alpha=0.05)

start = int(df_jp.index.max()) + 1
idx_fc = np.arange(start, start + steps)

fc = pd.DataFrame(fc_mean, index=idx_fc, columns=df_jp.columns)
lo = pd.DataFrame(fc_lo,   index=idx_fc, columns=df_jp.columns).add_suffix('_lo')
hi = pd.DataFrame(fc_hi,   index=idx_fc, columns=df_jp.columns).add_suffix('_hi')

# Plot
fig, ax = plt.subplots(figsize=(10,6))
df_jp[['yield_10y','inflation_yoy']].plot(ax=ax)
ax.plot(fc.index, fc['yield_10y'], '--', label='yield_10y (fcst)')
ax.fill_between(fc.index, lo['yield_10y_lo'], hi['yield_10y_hi'], alpha=0.2)
ax.plot(fc.index, fc['inflation_yoy'], '--', label='inflation_yoy (fcst)')
ax.fill_between(fc.index, lo['inflation_yoy_lo'], hi['inflation_yoy_hi'], alpha=0.2)
ax.axvline(df_jp.index.max(), ls='--', color='gray', alpha=0.6)
ax.set_title(f"Japón (2010–{df_jp.index.max()}) – VAR forecast ({steps} años)")
ax.set_ylabel('%'); ax.grid(True); ax.legend(); plt.tight_layout(); plt.show()

# 3) FEVD, Granger e IRFs
fevd = res_jp.fevd(10); print(fevd.summary()); fevd.plot(figsize=(9,5)); plt.show()
print(res_jp.test_causality('yield_10y', ['inflation_yoy'], kind='f').summary())
print(res_jp.test_causality('inflation_yoy', ['yield_10y'], kind='f').summary())
irf = res_jp.irf(10); irf.plot(orth=True); plt.show(); irf.plot_cum_effects(orth=True); plt.show()


### Resultados clave

Orden elegido (AIC): 4 → El modelo usa hasta 4 rezagos para explicar la dinámica.

Estabilidad VAR: False → 🚨 Esto significa que el sistema no cumple la condición de estabilidad (al menos una raíz característica >1).
En la práctica → el forecast es válido pero menos fiable, las bandas de error se disparan (como ves en la zona naranja).

### Interpretación del gráfico

Yield 10 años (línea azul)

Históricamente muy estable (alrededor del 1.5%–2%).

El forecast (línea verde discontinua) se mantiene casi plano, con poca variación.

→ El VAR refleja que Japón no tiene grandes movimientos en tipos largos.

Inflación interanual (línea naranja)

Mucha más volatilidad en la historia reciente (negativa en 2020, picos altos tras 2022).

El forecast (línea roja discontinua) muestra un rebote hacia arriba, con alta incertidumbre (bandas muy anchas).

→ Esto refleja que la inflación en Japón es muy difícil de prever con pocos datos.

### Conclusión corta para Japón

Los rendimientos a 10 años son muy estables, con pronóstico casi plano.

La inflación es altamente incierta, y el VAR la proyecta con posible aumento, pero con gran varianza.

La falta de estabilidad estadística sugiere que el modelo podría necesitar:

Más años de datos, o

Incluir más variables (ej. PIB, política monetaria, tipo de cambio).

## 🇩🇪 Alemania

Datos completos y estables.

Rendimientos y tipos muestran relación moderada.

Inflación relativamente contenida, forecast razonable.

## 🇺🇸 Estados Unidos

Serie más larga y robusta.

VAR bien estimado con lags pequeños.

Forecast estable: yields suben suavemente, inflación más volátil pero con señal clara.

## 🇪🇸 España

Serie corta pero consistente.

Rendimientos bajando tras 2010, inflación moderada.

Forecast: yields ligeramente al alza, inflación estable con bandas amplias.

## 🇬🇧 Reino Unido

Datos completos, VAR estable.

Rendimientos estables en torno a 4–5%.

Inflación muy volátil (pico 2022), forecast muestra normalización pero con incertidumbre.

🇯🇵 Japón

Serie con 15 observaciones → pocos datos.

Rendimientos extremadamente estables (1–2%).

Inflación impredecible: forecast incierto, modelo inestable.

Conclusión: se necesitan más variables para mejorar.

## 📌 Conclusión general

Robustos: USA y Alemania (mejor calidad de forecast).

Interesantes para comparar: UK y España (muestran volatilidad post-crisis e inflación reciente).

Frágil: Japón (modelo inestable, forecast poco fiable).

In [ ]:
# === ML Utilities (usamos para todos los países) ===

# 1) Dataset supervisado: lags como features
def make_supervised(df, lags=3, h=1, target='yield_10y'):
    Xy = df.copy()
    for L in range(1, lags+1):
        Xy[f'yield_lag{L}'] = Xy['yield_10y'].shift(L)
        Xy[f'infl_lag{L}']  = Xy['inflation_yoy'].shift(L)
    Xy['target'] = Xy[target].shift(-h)
    return Xy.dropna()

# 2) Expanding backtest con un modelo sklearn
def expanding_backtest(df, model, lags=3, h=1, test_start=2018):
    Xy = make_supervised(df, lags=lags, h=h)
    preds, trues, years = [], [], []
    for yr in Xy.index:
        if yr < test_start: 
            continue
        train = Xy.loc[Xy.index < yr]
        test  = Xy.loc[[yr]]
        if len(train) < 6: 
            continue
        X_tr, y_tr = train.drop(columns=['target']), train['target']
        X_te, y_te = test.drop(columns=['target']), test['target'].iloc[0]
        model.fit(X_tr, y_tr)
        y_hat = model.predict(X_te)[0]
        preds.append(y_hat); trues.append(y_te); years.append(int(yr))
    return pd.DataFrame({'Year': years, 'y_true': trues, 'y_pred': preds}).set_index('Year')

# 3) Baseline naive
def naive_forecast(df, h=1, start_year=2018):
    target = df['yield_10y'].shift(-h)
    naive  = df['yield_10y']
    out = pd.DataFrame({'y_true': target, 'y_pred': naive}).dropna()
    return out.loc[out.index >= start_year]

# 4) VAR baseline
def var_recursive_forecast(df, h=1, start_year=2018, maxlags=3):
    preds, trues, years = [], [], []
    for yr in range(start_year, int(df.index.max())+1):
        train = df.loc[df.index < yr]
        if len(train) < 6:
            continue
        safe = max(1, min(maxlags, (len(train)-3)//2))
        try:
            res = VAR(train).fit(ic='aic', maxlags=safe)
            y_hat = res.forecast(res.y, steps=1)[0][0]
            preds.append(y_hat)
            trues.append(df.loc[yr,'yield_10y'])
            years.append(yr)
        except Exception as e:
            print(f"skip {yr}: {e}")
            continue
    return pd.DataFrame({'Year': years, 'y_true': trues, 'y_pred': preds}).set_index('Year')

# 5) Métricas
def eval_metrics(df_pred):
    mae = mean_absolute_error(df_pred['y_true'], df_pred['y_pred'])
    rmse = sqrt(mean_squared_error(df_pred['y_true'], df_pred['y_pred']))
    return mae, rmse


In [ ]:
# === ML para USA ===
TEST_START = 2018

dfc = df_us[['yield_10y','inflation_yoy']].dropna().copy()
dfc.index = dfc.index.astype(int)

# Baseline Naive
p_naive = naive_forecast(dfc, h=1, start_year=TEST_START)
m_naive = eval_metrics(p_naive)

# VAR baseline
p_var = var_recursive_forecast(dfc, h=1, start_year=TEST_START, maxlags=3)
m_var = eval_metrics(p_var)

# ElasticNet
enet = ElasticNetCV(l1_ratio=[0.1,0.5,0.9], cv=3, max_iter=20000)
p_en  = expanding_backtest(dfc, enet, lags=3, h=1, test_start=TEST_START)
m_en  = eval_metrics(p_en)

# Random Forest
rf = RandomForestRegressor(n_estimators=400, random_state=42)
p_rf = expanding_backtest(dfc, rf, lags=3, h=1, test_start=TEST_START)
m_rf = eval_metrics(p_rf)

# Gradient Boosting
gb = GradientBoostingRegressor(random_state=42)
p_gb = expanding_backtest(dfc, gb, lags=3, h=1, test_start=TEST_START)
m_gb = eval_metrics(p_gb)

# Tabla resumen USA
usa_results = pd.DataFrame([
    ['Naive', *m_naive],
    ['VAR', *m_var],
    ['ElasticNet', *m_en],
    ['RandomForest', *m_rf],
    ['GradientBoosting', *m_gb],
], columns=['Model','MAE','RMSE'])

usa_results


In [ ]:
print(df.head())
print(df.tail())
print(df.shape)

In [ ]:
# Subset con las variables relevantes
dfc = df[['yield_10y', 'inflation_yoy']].dropna().copy()
dfc.index = dfc['Year'].astype(int)   # Usar el año como índice

# Probar naive forecast
p_naive = naive_forecast(dfc, h=1, start_year=2018)

print(p_naive.head())
print(p_naive.tail())
print("Shape:", p_naive.shape)


#### BLOQUE ML ROBUSTO

In [ ]:
# ---------- utilidades ----------
def make_supervised(df, lags=1):
    """
    Construye X,y con rezagos de yield_10y e inflation_yoy.
    y = yield_10y_t ; X = [yield_10y_{t-1..t-l}, infl_{t-1..t-l}]
    """
    work = df.copy()
    cols = []
    for k in range(1, lags+1):
        for c in ['yield_10y','inflation_yoy']:
            name = f"{c}_lag{k}"
            work[name] = work[c].shift(k)
            cols.append(name)
    work = work.dropna()
    X = work[cols].values
    y = work['yield_10y'].values
    idx = work.index.values.astype(int)
    return X, y, idx

def eval_metrics_safe(df_pred, label):
    if df_pred is None or len(df_pred)==0:
        return pd.Series({'Model':label, 'MAE':np.nan, 'RMSE':np.nan})
    mae = mean_absolute_error(df_pred['y_true'], df_pred['y_pred'])
    rmse = sqrt(mean_squared_error(df_pred['y_true'], df_pred['y_pred']))
    return pd.Series({'Model':label, 'MAE':mae, 'RMSE':rmse})

In [ ]:
# ---------- pronósticos por ventana expandida ----------
def naive_recursive_forecast(df, start_year):
    preds, trues, years = [], [], []
    for yr in range(start_year, int(df.index.max())+1):
        hist = df.loc[df.index < yr]
        if len(hist) < 1: 
            continue
        y_hat = float(hist['yield_10y'].iloc[-1])    # último valor
        y_true = float(df.loc[yr, 'yield_10y'])
        preds.append(y_hat); trues.append(y_true); years.append(int(yr))
    return pd.DataFrame({'Year':years, 'y_true':trues, 'y_pred':preds}).set_index('Year')

In [ ]:
def var_recursive_forecast_fix(df, start_year, p=1, min_train=3):
    preds, trues, years = [], [], []
    for yr in range(start_year, int(df.index.max())+1):
        train = df.loc[df.index < yr]
        if len(train) < max(min_train, p+1):
            continue
        try:
            res = VAR(train).fit(p)  # p fijo para muestras pequeñas
            preds.append(float(res.forecast(res.y, steps=1)[0][0]))  # 1ª col = yield
            trues.append(float(df.loc[yr, 'yield_10y']))
            years.append(int(yr))
        except Exception:
            continue
    return pd.DataFrame({'Year':years, 'y_true':trues, 'y_pred':preds}).set_index('Year')

In [ ]:
def sk_recursive_forecast(df, start_year, lags, model):
    X_all, y_all, idx = make_supervised(df, lags=lags)
    preds, trues, years = [], [], []
    # map year -> row position en X_all
    year_to_pos = {int(y):i for i,y in enumerate(idx)}
    for yr in range(start_year, int(df.index.max())+1):
        if yr not in year_to_pos: 
            continue
        pos = year_to_pos[yr]
        if pos < 1: 
            continue
        X_train, y_train = X_all[:pos], y_all[:pos]
        X_test, y_test   = X_all[pos:pos+1], y_all[pos:pos+1]
        if len(y_train) < 3:
            continue
        try:
            model.fit(X_train, y_train)
            y_hat = float(model.predict(X_test)[0])
            preds.append(y_hat); trues.append(float(y_test[0])); years.append(int(yr))
        except Exception:
            continue
    return pd.DataFrame({'Year':years, 'y_true':trues, 'y_pred':preds}).set_index('Year')

In [ ]:
# ---------- ejecución para un país ----------
def run_country_ML(dfc, country_name, test_start=2018, lags=1):
    dfc = dfc.dropna().copy()
    dfc.index = dfc.index.astype(int)
    if len(dfc) < max(6, lags+3):
        print(f"[{country_name}] muy pocos datos -> se omite.")
        return pd.DataFrame(columns=['Model','MAE','RMSE'])
    first, last = int(dfc.index.min()), int(dfc.index.max())
    print(f"{country_name} años disponibles: {first} → {last} | N= {len(dfc)}")
    TEST_START = max(test_start, first + lags + 2)  # garantía de mínimo train
    print("TEST_START usado:", TEST_START)

    # Naive
    p_naive = naive_recursive_forecast(dfc, TEST_START)
    m_naive = eval_metrics_safe(p_naive, "Naive")

    # VAR (p=1)
    p_var = var_recursive_forecast_fix(dfc, TEST_START, p=1, min_train=3)
    m_var  = eval_metrics_safe(p_var, "VAR(p=1)")

    # ElasticNet
    enet = Pipeline([('scaler', StandardScaler()),
                     ('model', ElasticNetCV(l1_ratio=[0.1,0.5,0.9], cv=3, max_iter=20000, n_jobs=None))])
    p_en = sk_recursive_forecast(dfc, TEST_START, lags=lags, model=enet)
    m_en = eval_metrics_safe(p_en, "ElasticNet")

    # RandomForest
    rf = RandomForestRegressor(n_estimators=400, max_depth=None, random_state=0)
    p_rf = sk_recursive_forecast(dfc, TEST_START, lags=lags, model=rf)
    m_rf = eval_metrics_safe(p_rf, "RandomForest")

    # GradientBoosting
    gb = GradientBoostingRegressor(random_state=0)
    p_gb = sk_recursive_forecast(dfc, TEST_START, lags=lags, model=gb)
    m_gb = eval_metrics_safe(p_gb, "GradientBoosting")

    res = pd.DataFrame([m_naive, m_var, m_en, m_rf, m_gb])
    print(f"# de predicciones usadas -> Naive: {len(p_naive)} | VAR: {len(p_var)} | EN: {len(p_en)} | RF: {len(p_rf)} | GB: {len(p_gb)}")
    return res

In [ ]:
# ---------- Lanza para los países disponibles ----------
all_results = []

countries = {
    'USA': 'df_us',
    'Germany': 'df_germany' if 'df_germany' in globals() else 'df_de',
    'Spain': 'df_es',
    'United Kingdom': 'df_uk',
    'Japan': 'df_jp'
}

for name, varname in countries.items():
    if isinstance(varname, str) and varname in globals():
        dfc = globals()[varname]
        try:
            dfc = dfc[['yield_10y','inflation_yoy']]
        except Exception:
            print(f"[{name}] dataframe no tiene columnas esperadas, se omite.")
            continue
        res = run_country_ML(dfc, name, test_start=2018, lags=1)
        if len(res):
            res.insert(0, 'Country', name)
            all_results.append(res)
    else:
        print(f"[{name}] no encontrado en el entorno, se omite.")

results_table = pd.concat(all_results, ignore_index=True) if all_results else pd.DataFrame(columns=['Country','Model','MAE','RMSE'])
display(results_table)
# ========= FIN BLOQUE =========


#### Backtest ML multi-país: yield_10y ~ {lags(yield_10y), lags(inflation_yoy)}

In [ ]:
# -------- 0) ENTRADAS ESPERADAS --------
# - DataFrame 'yields' con col 'time' (epoch ms o fecha) y columnas 10Y tipo 'US10','DE10','GB10','JP10','ES10',...
# - DataFrame 'wb_small' con columnas ['Country','Year','inflation_yoy']
#
# Si no existen en memoria, intenta cargarlos desde archivos habituales:
try:
    yields
except NameError:
    yields = pd.read_csv(YIELDS_CSV)

try:
    wb_small
except NameError:
    wb = pd.read_csv(WORLD_BANK_DATA)
    wb_small = wb.rename(columns={
        'country_name':'Country',
        'year':'Year',
        'Inflation (CPI %)':'inflation_yoy'
    })[['Country','Year','inflation_yoy']].dropna()

In [ ]:
# -------- 1) UTILIDADES DE DATOS --------
def make_annual_yields(yields_df: pd.DataFrame) -> pd.DataFrame:
    """Agrega por año todas las columnas XX10 que existan en 'yields'."""
    df = yields_df.copy()
    # Asegura columna fecha->año
    if 'time' in df.columns:
        try:
            df['time'] = pd.to_datetime(df['time'], unit='ms', errors='coerce')
        except Exception:
            df['time'] = pd.to_datetime(df['time'], errors='coerce')
        df = df.dropna(subset=['time'])
        df['Year'] = df['time'].dt.year
    elif 'Year' not in df.columns:
        raise ValueError("No encuentro 'time' ni 'Year' en yields.")
    # columnas 10Y (XX10)
    ten10 = [c for c in df.columns if re.fullmatch(r"[A-Z]{2}10", str(c))]
    if not ten10:
        raise ValueError("No encuentro columnas '*10' (p.ej. US10, DE10) en yields.")
    # media anual
    y_ann = df.groupby('Year', as_index=False)[ten10].mean()
    return y_ann

In [ ]:
def build_country_df(y_ann: pd.DataFrame, wb_small: pd.DataFrame, code2: str, country_name: str) -> pd.DataFrame:
    """Crea df con Year, yield_10y, inflation_yoy para un país."""
    col = f"{code2}10"
    if col not in y_ann.columns:
        raise KeyError(f"No existe {col} en yields anuales.")
    y = (y_ann[['Year', col]]
         .rename(columns={col:'yield_10y'}))
    i = (wb_small.query("Country == @country_name")[['Year','inflation_yoy']]
         .copy())
    df = (y.merge(i, on='Year', how='inner')
            .dropna()
            .set_index('Year')
            .sort_index())
    return df

In [ ]:
def make_lags(df: pd.DataFrame, lags:int=3) -> pd.DataFrame:
    """Crea variables rezagadas para ambos (yield_10y, inflation_yoy). Salida: X, y, index."""
    d = df.copy()
    for L in range(1, lags+1):
        d[f'yield_10y_l{L}'] = d['yield_10y'].shift(L)
        d[f'inflation_yoy_l{L}'] = d['inflation_yoy'].shift(L)
    d = d.dropna().copy()
    y = d['yield_10y'].copy()
    X = d.drop(columns=['yield_10y'])
    return X, y, d.index

In [ ]:
# -------- 2) MODELOS ROLLING --------
def naive_recursive(df: pd.DataFrame, start_year:int) -> pd.DataFrame:
    """Predicción naive: y_hat_t = y_{t-1}."""
    df = df.sort_index()
    preds, truth, idx = [], [], []
    years = df.index.values
    for t in years:
        if t < start_year: 
            continue
        pos = np.where(years==t)[0][0]
        if pos == 0: 
            continue
        y_hat = df.iloc[pos-1]['yield_10y']
        preds.append(float(y_hat))
        truth.append(float(df.iloc[pos]['yield_10y']))
        idx.append(int(t))
    return pd.DataFrame({'Year':idx,'y_pred':preds,'y_true':truth}).set_index('Year')

In [ ]:
def var_recursive(df: pd.DataFrame, start_year:int, maxlags:int=3) -> pd.DataFrame:
    """VAR rolling 1 paso; se salta si no hay suficientes datos."""
    df = df.sort_index()
    preds, truth, idx = [], [], []
    years = df.index.values
    for t in years:
        if t < start_year:
            continue
        end_pos = np.where(years==t)[0][0]
        train = df.iloc[:end_pos]
        if len(train) < (maxlags+4):  # seguridad
            continue
        safe = max(1, min(maxlags, (len(train)-2)//2))
        try:
            res = VAR(train).fit(ic='aic', maxlags=safe)
            y_hat = res.forecast(train.values[-res.k_ar:], steps=1)[0][0]  # 1ª variable = yield
        except Exception:
            continue
        preds.append(float(y_hat))
        truth.append(float(df.iloc[end_pos]['yield_10y']))
        idx.append(int(t))
    return pd.DataFrame({'Year':idx,'y_pred':preds,'y_true':truth}).set_index('Year')

In [ ]:
def sk_recursive(df: pd.DataFrame, start_year:int, model, lags:int=3) -> pd.DataFrame:
    """Framework común para ElasticNet / RF / GB con features de rezagos."""
    df = df.sort_index()
    # Pre-lags para no recalcular en cada ventana
    Xall, yall, idx_all = make_lags(df, lags=lags)  # índices coinciden con años válidos
    preds, truth, idx = [], [], []
    years = df.index.values
    for t in years:
        if t < start_year: 
            continue
        if t not in idx_all: 
            # no hay suficientes rezagos aún
            continue
        # train = años con índice < t dentro de idx_all
        mask_train = idx_all < t
        if mask_train.sum() < 5:  # mínimo razonable
            continue
        X_tr, y_tr = Xall[mask_train], yall[mask_train]
        try:
            mdl = model() if callable(model) else model
            mdl.fit(X_tr, y_tr)
            # pred para el año t (fila exacta de idx_all == t)
            x_t = Xall[idx_all==t]
            y_hat = mdl.predict(x_t)[0]
        except Exception:
            continue
        preds.append(float(y_hat))
        # verdad para t
        truth.append(float(df.loc[t, 'yield_10y']))
        idx.append(int(t))
    return pd.DataFrame({'Year':idx,'y_pred':preds,'y_true':truth}).set_index('Year')

In [ ]:
def metrics(df_pred: pd.DataFrame):
    if df_pred is None or df_pred.empty:
        return np.nan, np.nan, 0
    return (mean_absolute_error(df_pred['y_true'], df_pred['y_pred']),
            sqrt(mean_squared_error(df_pred['y_true'], df_pred['y_pred'])),
            len(df_pred))

In [ ]:
# -------- 3) PREPARAR Y CORRER --------
y_ann = make_annual_yields(yields)

countries = [
    # (nombre_mostrar, code2 en yields, nombre_en_WB)
    ("USA",     "US", "United States"),
    ("Germany", "DE", "Germany"),
    ("United Kingdom", "GB", "United Kingdom"),
    ("Japan",   "JP", "Japan"),
    ("Spain",   "ES", "Spain"),
]

TEST_START_DEFAULT = 2018   # para garantizar observaciones en la ventana de test
LAGS_SK = 3

rows = []

for display_name, code2, wb_name in countries:
    # construir panel país
    try:
        dfc = build_country_df(y_ann, wb_small, code2, wb_name)
    except Exception as e:
        print(f"[{display_name}] no se pudo construir el panel -> {e}. Se omite.")
        continue

    # rango y elección de test_start
    yr_min, yr_max, N = int(dfc.index.min()), int(dfc.index.max()), len(dfc)
    # si hay al menos 8–9 obs, usa 2018; si no, desplaza
    TEST_START = TEST_START_DEFAULT
    while TEST_START <= yr_min and TEST_START < yr_max:
        TEST_START += 1
    while (yr_max - TEST_START + 1) < 3 and TEST_START > yr_min:  # al menos 3 puntos de test
        TEST_START -= 1

    print(f"\n{display_name} años disponibles: {yr_min} → {yr_max} | N= {N}")
    print("TEST_START usado:", TEST_START)

    # NAIVE
    p_naive = naive_recursive(dfc, TEST_START)
    mae, rmse, n = metrics(p_naive)
    rows.append([display_name, "Naive", mae, rmse, n])

    # VAR (si alcanza)
    p_var = var_recursive(dfc, TEST_START, maxlags=3)
    mae, rmse, n = metrics(p_var)
    rows.append([display_name, f"VAR(p=? )", mae, rmse, n])

    # ElasticNet
    def EN(): return ElasticNetCV(l1_ratio=[0.1,0.5,0.9], cv=3, max_iter=30000, n_jobs=None)
    p_en = sk_recursive(dfc, TEST_START, EN, lags=LAGS_SK)
    mae, rmse, n = metrics(p_en)
    rows.append([display_name, "ElasticNet", mae, rmse, n])

    # RandomForest
    def RF(): return RandomForestRegressor(n_estimators=500, random_state=7)
    p_rf = sk_recursive(dfc, TEST_START, RF, lags=LAGS_SK)
    mae, rmse, n = metrics(p_rf)
    rows.append([display_name, "RandomForest", mae, rmse, n])

    # GradientBoosting
    def GB(): return GradientBoostingRegressor(random_state=7)
    p_gb = sk_recursive(dfc, TEST_START, GB, lags=LAGS_SK)
    mae, rmse, n = metrics(p_gb)
    rows.append([display_name, "GradientBoosting", mae, rmse, n])

    print(f"# de predicciones usadas => Naive: {len(p_naive)} | VAR: {len(p_var)} | EN: {len(p_en)} | RF: {len(p_rf)} | GB: {len(p_gb)}")

In [ ]:
# -------- 4) RESUMEN --------
summary = pd.DataFrame(rows, columns=['Country','Model','MAE','RMSE','N_pred']).sort_values(['Country','MAE'])
display(summary)
filename = "files/output/ml_backtest_summary.csv"
summary.to_csv(filename, index=False)
print(f"\nGuardado: {filename}")
